<a href="https://colab.research.google.com/github/zaku2590/signate/blob/main/%E3%82%B2%E3%83%BC%E3%83%A0%E3%81%AE%E5%8B%9D%E6%95%97%E4%BA%88%E6%B8%AC%E3%80%81%E3%83%AD%E3%82%B8%E3%82%B9%E3%83%86%E3%82%A3%E3%82%AF%E3%82%B9%E5%9B%9E%E5%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
sample_submit = pd.read_csv("sample_submit.csv", index_col=0, header=None)

train["blueKills-blueDeaths"] = train["blueFirstBlood"] + train["blueKills"] - train["blueDeaths"]
test["blueKills-blueDeaths"] = test["blueFirstBlood"] + test["blueKills"] - test["blueDeaths"]

train = train.drop(["blueFirstBlood"], axis=1)
test = test.drop(["blueFirstBlood"], axis=1)

train["blueEliteMonsters+blueDragons"] = train["blueEliteMonsters"] + train["blueDragons"]
test["blueEliteMonsters+blueDragons"] = test["blueEliteMonsters"] + test["blueDragons"]

train = train.drop(["blueKills", "blueDeaths", "blueEliteMonsters", "blueDragons"], axis=1)
test = test.drop(["blueKills", "blueDeaths", "blueEliteMonsters", "blueDragons"], axis=1)

先頭5行表示

In [ ]:
train.corrwith(train["blueWins"])

,0
blueAssists,0.311647
blueTotalGold,0.040489
blueTotalExperience,0.328167
blueWins,1.000000
blueKills-blueDeaths,0.396204
blueEliteMonsters+blueDragons,0.278769


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# 目的変数（y）と説明変数（X）の分離
y = train["blueWins"]  # 青チームが勝った場合1、それ以外は0
X = train.drop(["blueWins"], axis=1)  # blueWins以外の特徴量

# パイプライン作成（標準化 + ロジスティック回帰）
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 特徴量を標準化
    ('logreg', LogisticRegression(solver='lbfgs', max_iter=500))  # ロジスティック回帰モデル
])

# モデル学習
pipeline.fit(X, y)

# テストデータで予測（青チームが勝つ確率）
pred_proba = pipeline.predict_proba(test)[:, 1]  # ポジティブクラスの確率（青チームが勝つ確率）

# しきい値0.5を基準に0または1に変換
pred_binary = (pred_proba >= 0.5).astype(int)

# 提出用データの作成
# sample_submit["blueWins"] = pred_proba  # 確率で提出する場合
sample_submit = pd.DataFrame(index=test.index)
sample_submit["blueWins"] = pred_binary  # 0または1で提出する場合

# CSVに保存
sample_submit.to_csv("submission.csv", index=True, header=False)  # 提出用ファイル保存

# 結果の確認
print("予測確率 (最初の5件):", pred_proba[:5])  # 確率の確認
print("予測結果 (最初の5件):", pred_binary[:5])  # 0または1の確認

予測確率 (最初の5件): [0.4815229  0.59167153 0.40369718 0.35225491 0.60314287]
予測結果 (最初の5件): [0 1 0 0 1]
